In [ ]:
import os
import sys
import glob
import pickle
import tables

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
from comet_ml.api import API, APIExperiment
from comet_ml.query import Tag

if '..' not in sys.path:
    sys.path.append('..')
from deep_utils import *

#### A couple of functions used in the following

In [ ]:
def load_data(data_files, var_names, data_mean = None, data_std = None, window_dur = 60, window_step = 10, verbose=False):
    fids = [tables.open_file(data_file, 'r') for data_file in data_files]
    n_files = len(data_files)
    time = [fids[0].root.time.read()]
    for i in range(1, n_files):
        time.append(fids[i].root.time.read() + time[i-1][-1])
    time = np.concatenate(time)
    N_samples = time.size
    data = {var_name: np.concatenate([fid.root[var_name] for fid in fids]) for var_name in var_names}
    if data_mean is None:
        data_mean = {var_name: np.mean(data[var_name]) for var_name in var_names}
        print(f'data_mean = {data_mean}')
    if data_std is None:
        data_std = {var_name: np.std(data[var_name]) for var_name in var_names}
        print(f'data_std = {data_std}')
    data_normalized = {var_name: (data[var_name] - data_mean[var_name]) / data_std[var_name] for var_name in var_names}
    params = fids[0].root.parameters.read()
    dt = 1 / params['frand'][0]
    window_size = int(window_dur / dt)
    if verbose:
        print('Window size: {:d} samples'.format(window_size))
    data_sliding = {}
    indexes = {}
    for var_name in var_names:
        data_sliding[var_name], indexes[var_name] = slide_window(data_normalized[var_name],
                                                                  window_size,
                                                                  window_step=window_step)
    if verbose:
        print('Number of trials: {:d}'.format(data_sliding[var_names[0]].shape[0]))

    for fid in fids:
        fid.close()
    
    return time, data, data_normalized, data_sliding, indexes


def predict(model, data_sliding, window_step, dt, rolling_length=50):
    var_names = data_sliding.keys()
    if len(model.inputs) > 1:
        x = {var_name: tf.constant(data_sliding[var_name], dtype=tf.float32) for var_name in var_names}
    elif len(model.inputs) == 1 and len(data_sliding.keys()) > 1:
        x = tf.constant(list(data_sliding.values()), dtype=tf.float32)
        x = tf.transpose(x, perm=(1,2,0))
    else:
        x = tf.constant(data_sliding[var_names[0]], dtype=tf.float32)
    y = model.predict(x)
    n_samples, n_outputs = y.shape
    data = {f'inertia_{i}': y[:,i] for i in range(n_outputs)}
    H = pd.DataFrame(data).rolling(rolling_length).mean().to_numpy()
    time = np.arange(n_samples) * window_step * dt
    return time, H, y

#### Find the best experiment given a set of tags

In [ ]:
api = API(api_key = os.environ['COMET_API_KEY'])
workspace = 'danielelinaro'
project_name = 'inertia'
generator_IDs = [1, 2]
D = 2
DZA = 60
query = Tag('_'.join([f'G{gen_id}' for gen_id in generator_IDs])) & \
        Tag(f'D={D}') & \
        Tag(f'DZA={DZA}') & \
        Tag('1D_pipeline')
experiments = api.query(workspace, project_name, query, archived=False)
experiment_IDs = []
MAPE = []
val_loss = []
tags =  []
for experiment in experiments:
    ID = experiment.id
    experiment_IDs.append(ID)
    sys.stdout.write(f'Downloading data for experiment ID {ID}... ')
    metrics = experiment.get_metrics()
    sys.stdout.write('done.\n')
    val_loss.append(np.array([float(m['metricValue']) for m in metrics if m['metricName'] == 'val_loss']))
    has_MAPE = False
    for m in metrics:
        if m['metricName'] == 'mape_prediction':
            val = m['metricValue']
            try:
                MAPE.append(float(val))
            except:
                MAPE.append(list(map(float, val[1:-1].split(' ')[:2])))
            has_MAPE = True
            break
    tags.append(experiment.get_tags())
    print(f'  val_loss: {val_loss[-1].min():.4f}')
    if has_MAPE:
        print(f'      MAPE: {MAPE[-1]}%')
    else:
        print('      MAPE: [experiment not terminated]')
    print('      Tags: "{}"'.format('" "'.join(tags[-1])))
# idx = np.argmin(MAPE)
idx = np.argmin([loss.min() for loss in val_loss])
experiment_ID = experiment_IDs[idx]
MAPE = MAPE[idx]
val_loss = val_loss[idx]
tags = tags[idx]
print(f'The best experiment is {experiment_ID[:6]} (val_loss = {val_loss.min():.4f}, MAPE = {MAPE}%).')

In [ ]:
experiments_path = '../experiments/'
checkpoint_path = experiments_path + experiment_ID + '/checkpoints/'
checkpoint_files = glob.glob(checkpoint_path + '*.h5')
network_parameters = pickle.load(open(experiments_path + experiment_ID \
                                      + '/parameters.pkl', 'rb'))
epochs = [int(os.path.split(file)[-1].split('.')[1].split('-')[0]) for file in checkpoint_files]
best_checkpoint = checkpoint_files[epochs.index(np.argmin(val_loss) + 1)]
model = keras.models.load_model(best_checkpoint, compile=True)
try:
    data_dirs = ['../' + network_parameters['data_dir']]
except:
    data_dirs = ['../' + data_dir.format(gen_id) for gen_id in network_parameters['generator_IDs'] \
                 for data_dir in network_parameters['data_dirs']]
# we need mean and standard deviation of the training set to normalize the data
x_train_mean = network_parameters['x_train_mean']
x_train_std  = network_parameters['x_train_std']
if not os.path.isdir(data_dirs[0]):
    data_dirs[0] = '../data/var_H_G1/' + os.path.split(data_dirs[0])[-1]
print('Loaded network from {}.'.format(best_checkpoint))
print('Data directories are {}.'.format(data_dirs))

In [ ]:
keras.utils.plot_model(model, show_shapes=False, dpi=96)

### Constant inertia
Estimation of the value of inertia for all the generators in the IEEE14 network. The neural network has been trained using data from the generators in the <tt>generator_IDs</tt> list.

In [ ]:
generator_IDs_to_predict = [1, 6]
generator_IDs_map = {1: 1, 6: 2}
window_dur = 60
window_step = 10
H_value = 8.5
idx = 0
data_file = f'{data_dirs[idx]}/H_{H_value:.3f}.h5'
var_names = [var_name.format(gen_id) for gen_id in generator_IDs_to_predict \
             for var_name in network_parameters['var_names']]
if np.all(generator_IDs_to_predict == generator_IDs):
    data_mean = {var_name: x_train_mean[i] for i,var_name in enumerate(var_names)}
    data_std = {var_name: x_train_std[i] for i,var_name in enumerate(var_names)}
else:
    data_mean = {}
    data_std = {}
    for gen_id in generator_IDs_to_predict:
        for i,var_name_fmt in enumerate(network_parameters['var_names']):
            var_name = var_name_fmt.format(gen_id)
            j = (generator_IDs_map[gen_id] - 1) * 2 + i
            data_mean[var_name] = x_train_mean[j]
            data_std[var_name] = x_train_std[j]
    data_mean = None
#     data_std = None
t, data, data_normalized, data_sliding, _ = load_data([data_file],
                                                       var_names,
                                                       data_mean,
                                                       data_std,
                                                       window_dur,
                                                       window_step,
                                                       verbose=True)

if not np.all(generator_IDs_to_predict == generator_IDs):
    for src,dst in generator_IDs_map.items():
        for var_name in network_parameters['var_names']:
            data_sliding[var_name.format(dst)] = data_sliding.pop(var_name.format(src))

dt = np.diff(t[:2])[0]
time, inertia, _ = predict(model, data_sliding, window_step, dt)

In [ ]:
default_H = {
    1: 10.296 / 2,
    2: 13.08 / 2,
    3: 13.08 / 2,
    6: 10.12 / 2,
    8: 10.12 / 2
}

fig,ax = plt.subplots(1, 1, figsize=[10,5])
cmap = {1: 'k', 2: 'g', 3: 'm', 6: 'r', 8: 'c'}
for i,gen_id in enumerate(generator_IDs_to_predict):
    ax.plot(time / 60, inertia[:,i], label=f'G{gen_id}', color=cmap[gen_id], lw=1)
    if gen_id == generator_IDs[idx]:
        ax.plot(time[[0,-1]] / 60, H_value + np.zeros(2), '--', lw=1, color=cmap[gen_id])
    else:
        ax.plot(time[[0,-1]] / 60, default_H[gen_id] + np.zeros(2), '--', lw=1, color=cmap[gen_id])
ax.legend(loc='best')
ax.set_xlabel('Time [min]')
ax.set_ylabel(r'Inertia [s$^{-1}$]')
ax.set_xticks(np.arange(time[0], time[-1]/60+10, 30))
for side in 'right', 'top':
    ax.spines[side].set_visible(False)
# ax.set_ylim([5,10])
fig.tight_layout()

### Step of inertia
1. H steps **instantaneously** from 3.5 to 3.8
1. **omega** and **Pe** of the first generator used for the estimation
1. two simulations joined together (i.e., **no transient** during the step)

### Ramp of inertia
H increases **gradually** from 3.5 to 6.5 in 100 seconds (from t = 3550s to t=3650s)